In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


2025-08-19 15:01:03.821319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755608463.843521   14425 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755608463.850853   14425 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755608463.865919   14425 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755608463.865939   14425 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755608463.865941   14425 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Importy

In [3]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model


## Wczytanie modelu

In [4]:
model = load_model('../../model/2025-08-18/lstm.keras')

I0000 00:00:1755608468.424181   14425 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1733 MB memory:  -> device: 0, name: NVIDIA GeForce MX150, pci bus id: 0000:01:00.0, compute capability: 6.1


## Wczytanie etykiet z pliku tekstowego


In [5]:
with open('../../model/2025-07-14/labels.txt', 'r') as f:
    label_names = [line.strip() for line in f.readlines()]

## MediaPipe inicjalizacja

In [6]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1755608471.974249   14544 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755608471.997067   14544 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


# Bufor sekwencji

In [7]:
sequence = []
SEQ_LENGTH = 30
THRESHOLD = 0.8

## Normalizacja względem nadgarstka pierwszej klatki

In [8]:
def normalize_sequence(sequence_array):
    sequence_array = np.array(sequence_array)
    origin = sequence_array[0][0]  # Nadgarstek 1. klatki
    return sequence_array - origin

## Ekstrakcja 21 punktów prawej dłoni

In [1]:
def extract_hand_landmarks(hand_landmarks):
    if hand_landmarks:
        return np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
    return None


# Kamera na żywo

In [2]:
# Kamera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Rysowanie dłoni
            mp_drawing.draw_landmarks(frame,
                                      hand_landmarks,
                                      mp_hands.HAND_CONNECTIONS)

            # Ekstrakcja landmarków (musisz mieć własną funkcję extract_hand_landmarks dostosowaną do mp_hands)
            landmarks = extract_hand_landmarks(hand_landmarks)

            if landmarks is not None:
                sequence.append(landmarks)

                if len(sequence) == SEQ_LENGTH:
                    norm_seq = normalize_sequence(sequence)
                    input_seq = norm_seq.reshape(1, SEQ_LENGTH, 63)

                    pred = model.predict(input_seq)[0]
                    max_prob = np.max(pred)
                    label = label_names[np.argmax(pred)]

                    if max_prob > THRESHOLD:
                        cv2.putText(frame, f'{label} ({max_prob:.2f})', (10, 40),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    sequence.pop(0)
    else:
        sequence = []

    cv2.imshow('Real-Time Sign Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

# Kamera na żywo z cooldown'em

## Parametry

In [1]:
sequence = []
recognized_text = ""
last_prediction_time = 0
cooldown_seconds = 3
SEQ_LENGTH = 30
THRESHOLD = 0.9

In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Inicjalizacja MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.7)

# Kamera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Rysowanie dłoni
            mp_drawing.draw_landmarks(frame,
                                      hand_landmarks,
                                      mp_hands.HAND_CONNECTIONS)

            # Ekstrakcja landmarków (musisz mieć własną funkcję extract_hand_landmarks dostosowaną do mp_hands)
            landmarks = extract_hand_landmarks(hand_landmarks)

            if landmarks is not None:
                sequence.append(landmarks)

                if len(sequence) == SEQ_LENGTH:
                    norm_seq = normalize_sequence(sequence)
                    input_seq = norm_seq.reshape(1, SEQ_LENGTH, 63)

                    pred = model.predict(input_seq)[0]
                    max_prob = np.max(pred)
                    label = label_names[np.argmax(pred)]

                    if max_prob > THRESHOLD:
                        cv2.putText(frame, f'{label} ({max_prob:.2f})', (10, 40),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    sequence.pop(0)
    else:
        sequence = []

    cv2.imshow('Real-Time Sign Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


2025-08-19 15:03:40.683790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755608620.699163   14665 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755608620.703522   14665 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755608620.715270   14665 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755608620.715292   14665 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755608620.715293   14665 computation_placer.cc:177] computation placer alr

NameError: name 'extract_hand_landmarks' is not defined